In [1]:
import sqlite3
import os
import pandas as pd
import json
import numpy as np
from datetime import datetime
from dateutil.parser import parse
from plotly import graph_objects as go
from plotly import io as pio
from components import text_handling
import xmltodict
from components.api_tools.annotation import intact
from components.api_tools.annotation import biogrid

In [2]:
datadir = os.path.join('data_assets','db build files')
dbdir = 'data'
crapome = pd.read_csv(os.path.join(datadir,'crapome table.tsv'),sep='\t')
controls = pd.read_csv(os.path.join(datadir,'control table.tsv'),sep='\t')
jsons = {}
for f in os.listdir(datadir):
    if f.split('.')[-1]=='json':
        with open(os.path.join(datadir,f)) as fil:
            jsons[f'data_{f}'] = json.load(fil)

In [3]:
controls.head()

,PROTID,AP_GFP_C_1_S2-A2_1_6530,AP_GFP_C_2_S2-A3_1_6531,AP_GFP_C_3_S2-A4_1_6532,AP_GFP_MED_NLS_1_S2-B2_1_6542,AP_GFP_MED_NLS_2_S2-B3_1_6543,AP_GFP_MED_NLS_3_S2-B4_1_6544,AP_GFP_MYC_NLS_1_S2-B6_1_6546,AP_GFP_MYC_NLS_2_S2-B7_1_6547,AP_GFP_MYC_NLS_3_S2-B8_1_6548,...,SUC_N_GFP_BIO10_S1-C2_1_4443,SUC_N_GFP_BIO1_S3-H1_1_3637,SUC_N_GFP_BIO2_S3-H2_1_3638,SUC_N_GFP_BIO3_S3-H3_1_3639,SUC_N_GFP_BIO4_S3-H4_1_3640,SUC_N_GFP_BIO5_S3-H5_1_3641,SUC_N_GFP_BIO6_S3-H6_1_3642,SUC_N_GFP_BIO7_S3-H7_1_3643,SUC_N_GFP_BIO8_S3-H8_1_3644,SUC_N_GFP_BIO9_S1-C1_1_4442
0,A0A075B6H7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0A0A0MS14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0A0B4J1V0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0A0B4J1V1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A0A0B4J2D5,6.0,5.0,7.0,NaN,NaN,1.0,7.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
sets = {
    'VL GFP MAC3 AP': [
             'VL GFP MAC3-N AP-MS',
    ],
    'VL GFP MAC3 BioID': [
             'VL GFP MAC3-N BioID'
    ],
    'VL GFP MAC2 AP': [
             'VL GFP MAC2-C AP-MS',
             'VL GFP MAC2-N AP-MS',
    ],
    'VL GFP MAC2 BioID': [
             'VL GFP MAC2-C BioID',
             'VL GFP MAC2-N BioID'
    ],
    'VL GFP MAC AP': [
            'VL GFP MAC-C AP-MS',
            'VL GFP MAC-N AP-MS',
    ],
    'VL GFP MAC AP NLS': [
            'VL GFP MAC-MED-NLS AP-MS',
            'VL GFP MAC-MYC-NLS AP-MS',
            'VL GFP MAC-NLS AP-MS',
    ],
    'VL GFP MAC BioID': [
            'VL GFP MAC-C BioID',
            'VL GFP MAC-N BioID'
    ],
    'VL GFP MAC BioID NLS': [
            'VL GFP MAC-MED-NLS BioID',
            'VL GFP MAC-MYC-NLS BioID',
            'VL GFP MAC-NLS BioID'
    ],
    'Nesvilab': ['nesvilab']
}
crapome_tables = {}
columns = [
    'protein_id',
    'identified_in',
    'frequency',
    'spc_sum',
    'spc_avg',
    'spc_min',
    'spc_max',
    'spc_stdev']
types = [
    'TEXT PRIMARY KEY',
    'INTEGER NOT NULL',
    'REAL NOT NULL',
    'INTEGER NOT NULL',
    'REAL NOT NULL',
    'INTEGER NOT NULL',
    'INTEGER NOT NULL',
    'REAL NOT NULL'
]
crapome_entries = []
for setname, setcols in sets.items():
    all_cols = ['PROTID']
    defa = 1
    if 'MAC2' in setname: defa = 0 # default enabled value
    tablename = f'crapome_{setname}'.lower().replace(' ','_')
    for sc in setcols:
        all_cols.extend(jsons['data_crapome sets.json'][sc])
    all_cols = sorted(list(set(all_cols)))
    set_df = crapome[all_cols]
    setname = f'{setname} ({len(all_cols)} runs)'
    set_df.index = set_df['PROTID']
    set_df = set_df.drop(columns=['PROTID']).replace(0,np.nan).dropna(how='all',axis=0).dropna(how='all',axis=1)
    nruns = set_df.shape[1]
    set_data = []
    for protid, row in set_df.iterrows():
        stdval = row.std()
        if pd.isna(stdval):
            stdval = -1
        set_data.append([protid, row.notna().sum(), row.notna().sum()/nruns,row.sum(), row.mean(), row.min(), row.max(), stdval])
    crapome_tables[tablename] = pd.DataFrame(columns=columns, data=set_data)
    crapome_entries.append([tablename, setname, nruns, 0, defa, tablename])

In [4]:
control_tables = {}
control_entries = []
for setname, setcols in sets.items():
    if setname == 'Nesvilab': continue
    all_cols = ['PROTID']
    defa = 1
    if 'MAC2' in setname: defa = 0
    tablename = f'control_{setname}'.lower().replace(' ','_')
    for sc in setcols:
        all_cols.extend(jsons['data_control sets.json'][sc])
    all_cols = sorted(list(set(all_cols)))
    setname = f'{setname} ({len(all_cols)} runs)'
    set_df = controls[all_cols]
    set_df.index = set_df['PROTID']
    set_df = set_df.drop(columns=['PROTID']).replace(0,np.nan).dropna(how='all',axis=0).dropna(how='all',axis=1)
    nruns = set_df.shape[1]
    set_data = []
    for protid, row in set_df.iterrows():
        stdval = row.std()
        if pd.isna(stdval):
            stdval = -1
        set_data.append([protid, row.notna().sum(), row.notna().sum()/nruns,row.sum(), row.mean(), row.min(), row.max(), stdval])
    control_tables[tablename] = (set_df, pd.DataFrame(columns=columns, data=set_data))
    control_entries.append([tablename, setname, nruns, 0, defa, tablename])

control_cols = ['control_set','control_set_name','runs','is_disabled','is_default','control_table_name']
crapome_cols = ['crapome_set','crapome_set_name','runs','is_disabled','is_default','crapome_table_name']
exts = ['TEXT PRIMARY KEY','TEXT NOT NULL','INTEGER NOT NULL','INTEGER NOT NULL','INTEGER NOT NULL','TEXT NOT NULL']

control_table_str =  [
        f'CREATE TABLE control_sets (',
    ]
for i, c in enumerate(control_cols):
    control_table_str.append(f'    {c} {exts[i]},',)
control_table_str = '\n'.join(control_table_str).strip(',')
control_table_str += '\n);'

crapome_table_str =  [
        f'CREATE TABLE crapome_sets (',
    ]
for i, c in enumerate(crapome_cols):
    crapome_table_str.append(f'    {c} {exts[i]},',)
crapome_table_str = '\n'.join(crapome_table_str).strip(',')
crapome_table_str += '\n);'

prot_cols = [
    'uniprot_id',
    'is_reviewed',
    'gene_name',
    'entry_name',
    'all_gene_names',
    'organism',
    'length',
    'sequence',
    'is_latest',
    'entry_source',
    'update_time'
]
prot_exts = [
    'TEXT PRIMARY KEY',
    'INTEGER NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'INTEGER NOT NULL',
    'TEXT NOT NULL',
    'INTEGER NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL'
]

prot_table_str =  [
        f'CREATE TABLE proteins (',
    ]
for i, c in enumerate(prot_cols):
    prot_table_str.append(f'    {c} {prot_exts[i]},',)
prot_table_str = '\n'.join(prot_table_str).strip(',')
prot_table_str += '\n);'

table_create_sql = [control_table_str, crapome_table_str, prot_table_str]

insert_sql = []

for vals in control_entries:
    tablename = vals[0]
    detailed, overall = control_tables[tablename]
    detailed.rename(
        columns={
            c: 'CS_'+text_handling.replace_accent_and_special_characters(c,'_')
            for c in detailed.columns
        },
        inplace=True
    )
    create_str = [
        f'CREATE TABLE {tablename}_overall (',
    ]
    for i, c in enumerate(overall.columns):
        create_str.append(f'    {c} {types[i]},',)
    create_str = '\n'.join(create_str).strip(',')
    create_str += '\n);'
    table_create_sql.append(create_str)
    add_str = [f'INSERT INTO control_sets ({", ".join(control_cols)}) VALUES ({", ".join(["?" for _ in control_cols])})', vals]
    insert_sql.append(add_str)
    for _, row in overall.iterrows():
        add_str = [f'INSERT INTO {tablename}_overall ({", ".join(overall.columns)}) VALUES ({", ".join(["?" for _ in overall.columns])})', tuple(row.values)]
        insert_sql.append(add_str)
    create_str = [
        f'CREATE TABLE {tablename} (',
    ]
    detailed = detailed.reset_index()
    detailed_control_types = ['TEXT PRIMARY KEY']
    for c in detailed.columns[1:]:
        detailed_control_types.append('REAL')
    for i, c in enumerate(detailed.columns):
        create_str.append(f'    {c} {detailed_control_types[i]},',)
    create_str = '\n'.join(create_str).strip(',')
    create_str += '\n);'
    table_create_sql.append(create_str)
    for _, row in detailed.iterrows():
        add_str = [f'INSERT INTO {tablename} ({", ".join(detailed.columns)}) VALUES ({", ".join(["?" for _ in detailed.columns])})', tuple(row.values)]
        insert_sql.append(add_str)
print('control:', len(insert_sql))

for vals in crapome_entries:
    tablename = vals[0]
    create_str = [
        f'CREATE TABLE {tablename} (',
    ]
    for i, c in enumerate(columns):
        create_str.append(f'    {c} {types[i]},',)
    create_str = '\n'.join(create_str).strip(',')
    create_str += '\n);'
    table_create_sql.append(create_str)
    add_str = [f'INSERT INTO crapome_sets ({", ".join(crapome_cols)}) VALUES({", ".join(["?" for _ in crapome_cols])})', vals]
    insert_sql.append(add_str)
    for _, row in crapome_tables[tablename].iterrows():
        add_str = [f'INSERT INTO {tablename} ({", ".join(columns)}) VALUES ({", ".join(["?" for _ in columns])})', tuple(row.values)]
        insert_sql.append(add_str)
print('crapome:',len(insert_sql))

control: 74150


In [7]:
uniprot_df = pd.read_csv(os.path.join(datadir,'uniprotkb_AND_reviewed_true_2023_09_04.tsv'),sep='\t',index_col = 'Entry')
uniprots = set(uniprot_df.index.values)
for protid, row in uniprot_df.iterrows():
    gn = row['Gene Names (primary)']
    if pd.isna(gn):
        gn = row['Entry Name']
    gns = row['Gene Names']
    if pd.isna(gns):
        gns = row['Entry Name']
    row = row.fillna('')
    data = [
        protid,
        int(row['Reviewed']=='reviewed'),
        gn,
        row['Entry Name'],
        gns,
        row['Organism'],
        row['Length'],
        row['Sequence'],
        1,
        'uniprot_initial_download',
        datetime.today().strftime('%Y-%m-%d')
    ]
    add_str = f'INSERT INTO proteins ({", ".join(prot_cols)}) VALUES ({", ".join(["?" for _ in prot_cols])})'
    insert_sql.append([add_str, data])
print('protein:', len(insert_sql))


protein: 711646


In [9]:

cont_cols = [
    'uniprot_id',
    'is_reviewed',
    'gene_name',
    'entry_name',
    'all_gene_names',
    'organism',
    'length',
    'sequence',
    'entry_source',
    'contamination_source',
    'update_time'
]
cont_exts = [
    'TEXT PRIMARY KEY',
    'INTEGER NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'INTEGER NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL'
]

conts = pd.read_csv(os.path.join(datadir,'contaminant_list.tsv'),sep='\t')
conts = conts[~conts['Uniprot ID'].isin(['P0C1U8','Q2FZL2'])]
dd = pd.read_csv(os.path.join(datadir,'idmapping_2023_09_11.tsv'),sep='\t')
for _,row in dd.iterrows():
    conts.loc[conts[conts['Uniprot ID']==row['Entry']].index,'Length'] = row['Length']
dd2 = pd.read_csv(os.path.join(datadir,'idmapping_2023_09_121.tsv'),sep='\t')
for _, row in dd2.iterrows():
    ctloc = conts[conts['Uniprot ID']==row['From']]
    conts.loc[ctloc.index, 'Sequence'] = row['Sequence']
    conts.loc[ctloc.index, 'Gene names'] = row['Gene Names']
    conts.loc[ctloc.index, 'Length'] = row['Length']
    conts.loc[ctloc.index, 'Status'] = row['Reviewed']

seqs = {entry: row['Sequence'] for entry, row in uniprot_df.iterrows()}
seq_col = []
for _,row in conts.iterrows():
    if row['Uniprot ID'] not in seqs:
        seq_col.append('')
    else:
        seq_col.append(seqs[row['Uniprot ID']])
conts['Sequence'] = seq_col
conts['Length'] = conts['Length'].fillna(1).astype(int)
for i, row in conts[conts['Gene names'].isna()].iterrows():
    conts.loc[i, 'Gene names'] = f'{row["Protein names"]}({row["Uniprot ID"]})'
conts['Organism'] = conts['Organism'].fillna('None')
conts['Sequence'] = conts['Sequence'].fillna('Unknown')
conts['Sequence'] = conts['Sequence'].fillna('Unknown')
conts['Source of Contamination'] = conts['Source of Contamination'].fillna('Unspecified')
cont_table_str =  [
    f'CREATE TABLE contaminants (',
]
for i, c in enumerate(cont_cols):
    cont_table_str.append(f'    {c} {cont_exts[i]},',)
cont_table_str = '\n'.join(cont_table_str).strip(',')
cont_table_str += '\n);'
for _, row in conts.iterrows():
    gn = row['Gene names']
    if not 'Uncharac' in gn:
        gn = gn.split()[0]
    gns = row['Gene names']
    data = [
        row['Uniprot ID'],
        int(row['Status']=='reviewed'),
        gn,
        row['Entry name'],
        gns,
        row['Organism'],
        row['Length'],
        row['Sequence'],
        row['DataBase'],
        row['Source of Contamination'],
        datetime.today().strftime('%Y-%m-%d')
    ]
    add_str = f'INSERT INTO contaminants ({", ".join(cont_cols)}) VALUES ({", ".join(["?" for _ in cont_cols])})'
    insert_sql.append([add_str, data])
table_create_sql.append(cont_table_str)
print('contaminants:',len(insert_sql))

contaminants: 712093


In [10]:
tictable_create = ['CREATE TABLE IF NOT EXISTS ms_runs (']
tic_cols = [
    'run_id TEXT PRIMARY KEY',
    'sample_name TEXT NOT NULL',
    'run_name TEXT NOT NULL',
    'run_time TEXT NOT NULL',
    'instrument TEXT NOT NULL',
    'author TEXT NOT NULL',
    'sample_type TEXT NOT NULL',
    'run_type TEXT NOT NULL',
    'lc_method TEXT NOT NULL',
    'ms_method TEXT NOT NULL',
    'bait TEXT',
    'bait_uniprot TEXT',
    'chromatogram_max_time INTEGER NOT NULL',
    'cell_line TEXT',
    'project TEXT',
    'author_notes TEXT',
    'bait_tag TEXT',
]
for col in tic_cols:
    tictable_create.append(f'    {col},')
for col in  [
        'auc REAL NOT NULL',
        'intercepts INTEGER NOT NULL',
        'avg_peaks_per_timepoint REAL NOT NULL',
        'mean_intensity INTEGER NOT NULL',
        'max_intensity INTEGER NOT NULL',
        'json TEXT NOT NULL',
        'trace TEXT NOT NULL',
    ]:
    tic_cols.extend([
        f'bpc_{col}',
        f'msn_{col}',
        f'tic_{col}'
    ])
    tictable_create.extend([f'    {c},' for c in tic_cols[-3:]])
tictable_create = '\n'.join(tictable_create).strip(',')
tictable_create += '\n);'
table_create_sql.append(tictable_create)

## Run the dig_tic.py script at this point from an account with access to .d storage folder.

In [11]:
new_cols = []
for typ in ['BPC','TIC','MSn']:
    for key in ['auc','intercepts','peaks_per_timepoint','mean_intensity','max_intensity', 'json','trace', 'intercept_json']:
        new_cols.append(f'{typ}_{key}')
col_renames = {c: c.replace('peaks_per_timepoint','avg_peaks_per_timepoint') for c in new_cols}
tic_dir = os.path.join('data_assets','TIC data')
tic_data_files = [os.path.join(tic_dir, f) for f in os.listdir(tic_dir) if 'TIC data_' in f]
tic_data = pd.concat([pd.read_csv(f,sep='\t') for f in tic_data_files]).drop(columns='Unnamed: 0')
new_data = [[] for _ in new_cols]
for _,row in tic_data.iterrows():
    dic = json.loads(row['chromatograms'])
    if len(dic['json'].keys()) == 0:
        for i, nc in enumerate(new_cols):
            new_data[i].append(np.nan)
    else:
        for i, nc in enumerate(new_cols):
            typ, key = nc.split('_',maxsplit=1)
            new_data[i].append(dic[key][typ])
for i, nc in enumerate(new_cols):
    tic_data[nc] = new_data[i]
if not 'chromatogram_max_time' in tic_data.columns:
    tvals = []
    for _,row in tic_data.iterrows():
        tvals.append(pd.Series(row['BPC_json']).index.max())
    tic_data['chromatogram_max_time'] = tvals
dic_cols = [c for c in tic_data.columns if 'json' in c]
for d in dic_cols:
    nvals = [json.dumps(v) if type(v) is dict else str(v).replace('nan','') for v in tic_data[d].values]
    tic_data[d] = nvals
tic_data = tic_data.rename(columns=col_renames)
tic_data = tic_data.rename(columns={
    c: c.lower() for c in tic_data.columns
})
tic_data = tic_data.rename(columns={
    'sample_id': 'run_id', 'datafolder_name': 'run_name','HyStar_LC_Method_Name'.lower(): 'lc_method', 'HyStar_MS_Method_Name'.lower(): 'ms_method'
})
tic_data['instrument'] = 'timsTOF pro 2'
tic_data['run_id'] = tic_data['run_id'].astype(int)
runlist = pd.read_excel(os.path.join('..','..','..','combined runlist.xlsx'))
nvals = {c: [] for c in [r for r in tic_cols if r.split()[0] not in tic_data.columns]}
for _,row in tic_data.iterrows():
    run = runlist[runlist['Raw file']==row['run_name']]
    if run.shape[0]==0:
        for key in nvals.keys():
            nvals[key].append('')
    else:
        run = run.iloc[0]
        nvals['author TEXT NOT NULL'].append(run['Who'])
        nvals['sample_type TEXT NOT NULL'].append(run['Sample type'])
        nvals['run_type TEXT NOT NULL'].append(run['Run type'])
        nvals['cell_line TEXT'].append(run['Cell line / material'])
        nvals['project TEXT'].append(run['Project'])
        nvals['author_notes TEXT'].append(run['Notes'])
        nvals['bait_tag TEXT'].append(run['tag'])
        nvals['bait TEXT'].append(run['Bait / other uniprot or ID'])
        if run['Bait / other uniprot or ID'] in uniprots:
            nvals['bait_uniprot TEXT'].append(run['Bait / other uniprot or ID'])
        else:
            nvals['bait_uniprot TEXT'].append('')
for c, vals in nvals.items():
    tic_data[c.split()[0]] = vals
tic_data = tic_data[tic_data['chromatogram_max_time']!='']
tic_data = tic_data[tic_data['chromatogram_max_time']!=0]
tic_data = tic_data[tic_data['chromatogram_max_time'].notna()]
tic_data = tic_data.drop_duplicates()
print(len(insert_sql))
for _,row in tic_data.iterrows():
    data = [
        row[c.split()[0]] for c in tic_cols
    ]
    add_str = f'INSERT INTO ms_runs ({", ".join([c.split()[0] for c in tic_cols])}) VALUES ({", ".join(["?" for _ in tic_cols])})'
    insert_sql.append([add_str, data])
print(len(insert_sql))

In [15]:
inttable_create = ['CREATE TABLE IF NOT EXISTS known_interactions (']
inttable_cols = [
    'interaction TEXT PRIMARY KEY',
    'uniprot_id_a TEXT NOT NULL',
    'uniprot_id_b TEXT NOT NULL',
    'uniprot_id_a_noiso TEXT NOT NULL',
    'uniprot_id_b_noiso TEXT NOT NULL',
    'source_database TEXT NOT NULL',
    'isoform_a TEXT',
    'isoform_b TEXT',
    'experimental_role_interactor_a TEXT',
    'interaction_detection_method TEXT',
    'publication_identifier TEXT',
    'biological_role_interactor_b TEXT',
    'annotation_interactor_a TEXT',
    'confidence_value TEXT',
    'interaction_type TEXT',
    'experimental_role_interactor_b TEXT',
    'annotation_interactor_b TEXT',
    'biological_role_interactor_a TEXT',
    'publication_count TEXT',
    'notes TEXT',
    'update_time TEXT',
]
for col in inttable_cols:
    inttable_create.append(f'    {col},')
inttable_create = '\n'.join(inttable_create).strip(',')
inttable_create += '\n);'
table_create_sql.append(inttable_create)
print(len(insert_sql))

biogrid.update(uniprots)
intact.update(uniprots)
dbtables = [intact.get_latest(), biogrid.get_latest()]
for d in dbtables:
    if 'Unnamed: 0' in d.columns:
        d.drop(columns=['Unnamed: 0'],inplace = True)
shared = set()
for i, d in enumerate(dbtables):
    for d2 in dbtables[i+1:]:
        shared |= (set(d2.index) & set(d.index))

shared = sorted(list(shared))
ind = 0
if len(shared) > 0:
    mtables = [d.loc[shared].sort_index() for d in dbtables]
    dbtables = [d.drop(index=shared) for d in dbtables]
new_data = []
no_set = [c for c in mtables[0].columns if (('uniprot' not in c))]
for c in mtables[0].columns:
    if c == 'source_database':
        jst = ';'
    else:
        jst = '__'
    if c in no_set:
        nc = mtables[0][c].astype(str)
        for d in mtables[1:]:
            nc = nc + jst + d[c].astype(str)
        new_data.append(nc)
    else:
        new_data.append(mtables[0][c])
newer_data = [
    c.str.replace('nan','').str.strip('_') for c in new_data
]
shared_df = pd.DataFrame.from_dict({c: newer_data[i] for i, c in enumerate(mtables[0].columns)}).replace('__',np.nan)
shared_df.index = mtables[0].index
dbtables.append(shared_df)
merg = pd.concat(dbtables)
pm = []
npubs = []
for _,row in merg.iterrows():
    pmids = set()
    for p in row['publication_identifier'].split('__'):
        for pp in p.split(';'):
            if 'pubmed' in pp.lower():
                pmids.add(pp)
    pm.append(len(pmids))
    npubs.append(';'.join(sorted(list(pmids))))
merg['publication_count'] = pm
merg['publication_identifier'] = npubs

int_df_slim = merg[merg['uniprot_id_a'].isin(uniprots) & merg['uniprot_id_b'].isin(uniprots)]
int_df_slim = int_df_slim.reset_index()
for _,row in int_df_slim.iterrows():
    data = [
        row[c.split()[0]] for c in inttable_cols
    ]
    add_str = f'INSERT INTO known_interactions ({", ".join([c.split()[0] for c in inttable_cols])}) VALUES ({", ".join(["?" for _ in inttable_cols])})'
    insert_sql.append([add_str, data])
print('Knowns', len(insert_sql))


NameError: name 'prot' is not defined

In [28]:
msmictable_create = ['CREATE TABLE IF NOT EXISTS msmicroscopy (']
msmictable_cols = [
    'Interaction TEXT PRIMARY KEY',
    'Bait TEXT NOT NULL',
    'Prey TEXT NOT NULL',
    'Bait_norm REAL NOT NULL',
    'Bait_sumnorm REAL NOT NULL',
    'Loc TEXT NOT NULL',
    'Unique_to_loc REAL NOT NULL',
    'Loc_norm REAL NOT NULL',
    'Loc_sumnorm REAL NOT NULL',
    'MSMIC_version TEXT NOT NULL'
]
for col in msmictable_cols:
    msmictable_create.append(f'    {col},')
msmictable_create = '\n'.join(msmictable_create).strip(',')
msmictable_create += '\n);'
table_create_sql.append(msmictable_create)
print(len(insert_sql))
for dirname in os.listdir(os.path.join(datadir,'msmic')):
    if not os.path.isdir(os.path.join(datadir, 'msmic',dirname)):
        continue
    version = dirname
    ref_data = pd.read_csv(os.path.join(datadir, 'msmic', version, 'msmic_ref_table.txt'),sep='\t')
    loc_data = pd.read_csv(os.path.join(datadir, 'msmic', version, 'msmic_localizations.txt'),sep='\t')
    loc_col = 'Organelle'

    loc_data[loc_col] = [s.capitalize().strip() for s in loc_data[loc_col].values]
    baitnorm = []
    baitsumnorm = []
    preys_in_baits = {}
    preys_in_localizations = {}
    db_bait_max = {}
    db_bait_sum= {}
    for b in ref_data['Bait'].unique():
        db_bait_max[b] = max(ref_data[ref_data['Bait']==b]['AvgSpec'].values)
        db_bait_sum[b] = sum(ref_data[ref_data['Bait']==b]['AvgSpec'].values)
    for _,row in ref_data.iterrows():
        if row['Prey'] not in preys_in_baits:
            preys_in_baits[row['Prey']] = {}
            preys_in_localizations[row['Prey']] = {}
        preys_in_baits[row['Prey']][row['Bait']] = row['AvgSpec']
        baitnorm.append(row['AvgSpec']/db_bait_max[row['Bait']])
        baitsumnorm.append(row['AvgSpec']/db_bait_sum[row['Bait']])
        localization = loc_data[loc_data['Bait']==row['Bait']].iloc[0][loc_col]
        if localization not in preys_in_localizations:
            preys_in_localizations[row['Prey']][localization] = []
        preys_in_localizations[row['Prey']][localization].append(row['AvgSpec'])
    ref_data['Bait_norm'] = baitnorm    
    ref_data['Bait_sumnorm'] = baitsumnorm
    unique_preys = [p for p, v in preys_in_localizations.items() if len(v) == 1]
    ref_data['Loc'] = [loc_data[loc_data['Bait']==bait].iloc[0][loc_col] for bait in ref_data['Bait'].values]
    ref_data['Unique_to_loc'] = [prey in unique_preys for prey in ref_data['Prey'].values]

    uref = ref_data[ref_data['Unique_to_loc']].copy()
    locnorm = []
    locsumnorm = []
    loc_max = {}
    loc_sum = {}
    for l in uref['Loc'].unique():
        loc_max[l] = uref[uref['Loc']==l]['AvgSpec'].max()
        loc_sum[l] = uref[uref['Loc']==l]['AvgSpec'].sum()
    for _,row in uref.iterrows():
        locnorm.append(row['AvgSpec']/loc_max[row['Loc']])
        locsumnorm.append(row['AvgSpec']/loc_sum[row['Loc']])
    uref['Loc_norm'] = locnorm
    uref['Loc_sumnorm'] = locsumnorm
    uref['MSMIC_version'] = version
    uref['Interaction'] = uref['Bait']+uref['Prey']

    for _,row in uref.iterrows():
        data = [
            row[c.split()[0]] for c in msmictable_cols
        ]
        add_str = f'INSERT INTO msmicroscopy ({", ".join([c.split()[0] for c in msmictable_cols])}) VALUES ({", ".join(["?" for _ in msmictable_cols])})'
        insert_sql.append([add_str, data])
    print(version, len(insert_sql))

3178426
v1.0 3180029


In [29]:

# Connect to the database (create it if it doesn't exist)
conn = sqlite3.connect(os.path.join(dbdir,'proteogyver2.db'))
# Create a cursor object
cursor = conn.cursor()
start = datetime.now()
for create_table_str in table_create_sql:
    cursor.execute(create_table_str)
for insert_str, insert_data in insert_sql:
    cursor.execute(insert_str, insert_data)
print('Table creation and data insertion took', (datetime.now() - start).seconds, 'seconds')
# Commit changes and close the connection
conn.commit()
conn.close()


Table creation and data insertion took 12 seconds


In [31]:

con = sqlite3.connect(os.path.join(dbdir,'proteogyver.db'))
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
con.close()

[('control_sets',), ('crapome_sets',), ('proteins',), ('control_vl_gfp_mac3_ap_overall',), ('control_vl_gfp_mac3_ap',), ('control_vl_gfp_mac3_bioid_overall',), ('control_vl_gfp_mac3_bioid',), ('control_vl_gfp_mac2_ap_overall',), ('control_vl_gfp_mac2_ap',), ('control_vl_gfp_mac2_bioid_overall',), ('control_vl_gfp_mac2_bioid',), ('control_vl_gfp_mac_ap_overall',), ('control_vl_gfp_mac_ap',), ('control_vl_gfp_mac_ap_nls_overall',), ('control_vl_gfp_mac_ap_nls',), ('control_vl_gfp_mac_bioid_overall',), ('control_vl_gfp_mac_bioid',), ('control_vl_gfp_mac_bioid_nls_overall',), ('control_vl_gfp_mac_bioid_nls',), ('crapome_vl_gfp_mac3_ap',), ('crapome_vl_gfp_mac3_bioid',), ('crapome_vl_gfp_mac2_ap',), ('crapome_vl_gfp_mac2_bioid',), ('crapome_vl_gfp_mac_ap',), ('crapome_vl_gfp_mac_ap_nls',), ('crapome_vl_gfp_mac_bioid',), ('crapome_vl_gfp_mac_bioid_nls',), ('crapome_nesvilab',), ('contaminants',), ('ms_runs',), ('known_interactions',), ('msmicroscopy',)]
